In [1]:
from wdtools import *

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
set_ID = 'Set001'

In [3]:
#gdf = combine_merged_data(setID = set_ID)

it took 17.0 minutes to complete Set001


In [3]:
gdf = gpd.read_file(os.path.join(inpath + f'\\{outfolder}\\', f'matched_records_{set_ID}.shp'), driver='ESRI Shapefile')

In [4]:
gdf.shape

(2374, 30)

In [5]:
gdf.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'lots', 'lot', 'cnt_code', 'ORTaxlot', 'year', 'IDyear',
       'geometry', 'nrecordID'],
      dtype='object')

In [4]:
unmatched_wd_df = report_unmatched(gdf = gdf, setID = set_ID)

it is about 8.38% of data in the original 1420 records unmatched
there are 1 records (0.07% of the original records) without parcel id


In [5]:
missed_match_ID = compare_data_report(gdf = gdf, setID = set_ID)[0] #export = True

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\geodataframe.py:1299: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)


In [6]:
double_check = unmatched_wd_df[unmatched_wd_df.missinglot == 'N']

In [7]:
double_check.parcel_id.unique()

array(['3502,3800', '701', '12308',
       '3400,2900,2800,SW 2nd street ROW (all partial)', '501', '502',
       '8100', '1302', '101, 100', '101', '1700p', '300', '140p', '200p',
       '5001', '2900', '2100', '800,803', '400,500 all partial', '1401',
       '4400', '100', '400p', '1000', 'ROW, 6317,6303,3604', '400',
       '1403,1404', '2600', '302', '4900p', '500p', '100p', '105p', '708',
       '5100', '1400', '6702', '4500', '203', '5500 partial', '500',
       '3100', '2800', '500p,1100p', '2400,2401', '800', '3000'],
      dtype=object)

In [8]:
double_check_ri = reindex_data(double_check)

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [9]:
double_check_ri.head(3)

,wetdet_delin_number,trsqq,parcel_id,address_location_desc,city,county,site_name,site_desc,latitude,longitude,...,response_date,reissuance_response_date,project_id,site_id,record_ID,missinglot,lots,lot,cnt_code,ORTaxlot
16,WD2017-0313,08N07W20,"3502,3800",Hillcrest Lp and Forest Court Lp Astoria,Astoria,Clatsop,08N07W20,property evaluation for road and home construc...,46.16336,-123.577,...,2017-08-07,NaN,NaN,106418,17,N,"[3502, 3800]",3502,4,0408.00N07.00W2000--000003502
16,WD2017-0313,08N07W20,"3502,3800",Hillcrest Lp and Forest Court Lp Astoria,Astoria,Clatsop,08N07W20,property evaluation for road and home construc...,46.16336,-123.577,...,2017-08-07,NaN,NaN,106418,17,N,"[3502, 3800]",3800,4,0408.00N07.00W2000--000003800
21,WD2017-0448,08N10W20,701,W of 480 SW Juniper Ave,Warrenton,Clatsop,South Juniper Gardens 08N10W20,Subdivision and construction of single family ...,46.16377,-123.951,...,2017-12-07,NaN,NaN,107013,22,N,[701],701,4,0408.00N10.00W2000--000000701


In [12]:
def combine_taxlot():
    frames = []
    for year in range(2017, 2023):
        tx_dt = read_taxlot(year, mute=False)
        tx_dt['year'] = year
        frames.append(tx_dt[['year', 'ORTaxlot', 'geometry']])
    df = pd.concat(frames, ignore_index=True)
    gdf = gpd.GeoDataFrame(df, crs="EPSG:2992", geometry='geometry')
    return gdf

In [13]:
all_taxlot = combine_taxlot()

got taxlot data in 2017 and it took about 3.0 minutes
got taxlot data in 2018 and it took about 3.0 minutes
got taxlot data in 2019 and it took about 3.0 minutes
got taxlot data in 2020 and it took about 3.0 minutes
got taxlot data in 2021 and it took about 3.0 minutes
got taxlot data in 2022 and it took about 3.0 minutes


In [16]:
all_txid = all_taxlot.ORTaxlot.unique()

In [17]:
tocheck_txid = double_check_ri.ORTaxlot.unique()

In [18]:
len(tocheck_txid)

67

In [19]:
[txid for txid in tocheck_txid if txid in all_txid]

['0915.00S13.00E1800--000002900',
 '0915.00S13.00E0800--000000800',
 '0915.00S13.00E0800--000000803',
 '0915.00S13.00E04DB--000000100',
 '0917.00S12.00E22DA--000000101',
 '1323.00S25.00E0000--000000105',
 '1537.00S01.00W16D0--000006702',
 '2408.00S02.00W20D0--000000100']